In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime


In [93]:
#%% 0. Housekeeping 

# =============================================================================
# 0.1 Import packages
# =============================================================================

import pandas as pd
import datetime as dt
from datetime import datetime
   
# =============================================================================
# 0.2 Import data
# =============================================================================

train = pd.read_csv('data/train.csv', low_memory=False)
store = pd.read_csv('data/store.csv', low_memory=False)

#%% 1. Merging store to train data

# =============================================================================
# 1.1 Since data has to have the same size as 'train', a container is created
# =============================================================================

# Creating dataframe
expanded_store = train

# =============================================================================
# 1.2 Merging variables which do not have to be changed
# =============================================================================

# Variables which can be merged right away
easy = store.loc[:,['Store','StoreType','Assortment','CompetitionDistance']]

# Variables which can be simply merged
expanded_store = pd.merge(expanded_store, easy, on=['Store'])

#%% 2. Creating a dummy variable since for competition is open for every store

# =============================================================================
# 2.1 Checking whether there is actually competition
# =============================================================================

# Dataset of the variables which have to be transformed
competition = store.loc[:,['Store',
                           'CompetitionOpenSinceMonth','CompetitionOpenSinceYear']]

# For easier looping 
competition.set_index('Store', inplace=True)

# Finding stores which have competition openend
open_comp = competition.any(axis=1)

# Stores with competition
open_stores = competition.loc[open_comp].index.unique()

# =============================================================================
# 2.2 Generating dummy for the time competition is existing
# =============================================================================

# Generating Date when opened
for store_id in open_stores: 
    year = competition.loc[store_id,'CompetitionOpenSinceYear'].astype(int)
    month = competition.loc[store_id,'CompetitionOpenSinceMonth'].astype(int)
    competition.loc[store_id,'CompetitionDate'] = datetime(year,month,1)

# Creating a dummy variable for whether competition openend for each store
expanded_store.loc[:,'CompetitionOpened'] = 0

for store_id in open_stores: 
    
    # Getting the date when competition openend
    date = competition.loc[store_id,'CompetitionDate']
    date_str= date.strftime("%Y-%m-%d")
    
    store_number = expanded_store.loc[:,'Store'] == store_id
    
    maximum_date = expanded_store.loc[store_number,'Date'].max()
    
    # Whether it falls in time frame
    competition_existing = (expanded_store.loc[:,'Date'].between(date_str,maximum_date)) &  (expanded_store.loc[:,'Store'] == store_id)
    
    # Indicating whether competition is around
    expanded_store.loc[competition_existing, 'CompetitionOpened'] = 1


#%% 3. Creating a dummy variable for Promo
 
# =============================================================================
# 3.1 Since data has to have the same size as 'train', a container is created
# =============================================================================

# Create dataframe
promo2 = store.loc[:,['Store',
                     'Promo2','Promo2SinceWeek','Promo2SinceYear','PromoInterval']]

# =============================================================================
# 3.2 Create dummy for whether a promo2 is running 
# =============================================================================

# Getting date from which promo started
length = promo2.shape[0]
for i in range(length): 
    if promo2.loc[i,'Promo2']:
        week = promo2.loc[i,'Promo2SinceWeek'].astype(int)
        year = promo2.loc[i,'Promo2SinceYear'].astype(int)
        promo2.loc[i,'promo2start'] = dt.datetime.strptime(f'{year}-W{int(week )- 1}-1', "%Y-W%W-%w").date()

# Merge it with the train file
expanded_promo = pd.merge(expanded_store, promo2, on=['Store'])


# Empty container with no promo indicator
expanded_promo.loc[:,'Promo2GoingOn'] = 0

# Month indication
expanded_promo.loc[:,'Date_str'] = pd.to_datetime(expanded_promo.loc[:,'Date'],).dt.strftime('%Y-%b-%d')
expanded_promo.loc[:,'Date'] = pd.to_datetime(expanded_promo.loc[:,'Date'],)
expanded_promo.loc[:,'month'] = expanded_promo.loc[:,'Date_str'].str[5:8]


months = expanded_promo.loc[:,'month'].unique()

for month in months:
    month_boolean = expanded_promo.loc[:,'PromoInterval'].str.contains(month, na=False)
    expanded_promo.loc[month_boolean,'Promo2GoingOn'] = 1

In [97]:
expanded_promo.columns
expanded=expanded_promo
expanded.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpened', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'promo2start', 'Promo2GoingOn',
       'Date_str', 'month'],
      dtype='object')

In [98]:
# Convert Dates to Date time
expanded['Date']=pd.to_datetime(expanded['Date'])


In [99]:
#add in variables for day of month etc
expanded['dayofweek'] = expanded['Date'].dt.dayofweek
expanded['quarter'] = expanded['Date'].dt.quarter
expanded['month'] = expanded['Date'].dt.month
expanded['year'] = expanded['Date'].dt.year
expanded['dayofyear'] = expanded['Date'].dt.dayofyear
expanded['dayofmonth'] = expanded['Date'].dt.day
expanded['weekofyear'] = expanded['Date'].dt.weekofyear

In [100]:
#create dummy variables for day of week etc and categorical variables

expanded= pd.get_dummies(expanded,columns=['dayofweek','dayofmonth','quarter','month','StateHoliday'])

#expanded.drop(expanded['Date_str'])


In [101]:
#Re-add original day of month variable etc.
expanded['dayofweek'] = expanded['Date'].dt.dayofweek
expanded['quarter'] = expanded['Date'].dt.quarter
expanded['month'] = expanded['Date'].dt.month
expanded['year'] = expanded['Date'].dt.year
expanded['dayofyear'] = expanded['Date'].dt.dayofyear
expanded['dayofmonth'] = expanded['Date'].dt.day
expanded['weekofyear'] = expanded['Date'].dt.weekofyear




In [102]:
expanded.columns

Index(['Date', 'Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpened', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear',
       'PromoInterval', 'promo2start', 'Promo2GoingOn', 'Date_str', 'year',
       'dayofyear', 'weekofyear', 'dayofweek_0', 'dayofweek_1', 'dayofweek_2',
       'dayofweek_3', 'dayofweek_4', 'dayofweek_5', 'dayofweek_6',
       'dayofmonth_1', 'dayofmonth_2', 'dayofmonth_3', 'dayofmonth_4',
       'dayofmonth_5', 'dayofmonth_6', 'dayofmonth_7', 'dayofmonth_8',
       'dayofmonth_9', 'dayofmonth_10', 'dayofmonth_11', 'dayofmonth_12',
       'dayofmonth_13', 'dayofmonth_14', 'dayofmonth_15', 'dayofmonth_16',
       'dayofmonth_17', 'dayofmonth_18', 'dayofmonth_19', 'dayofmonth_20',
       'dayofmonth_21', 'dayofmonth_22', 'dayofmonth_23', 'dayofmonth_24',
       'dayofmonth_25', 'dayofmonth_26', 'dayofmonth_27', 'dayofmonth_28',
       'dayofmonth_29', 'dayofmonth_

In [103]:
#create average sales per customer
df=expanded.groupby(by=['Store']).agg({'Sales':'sum','Customers':'sum'})
df['av_SalesPerCustomer']=df['Sales']/df['Customers']
df.sort_values(by='av_SalesPerCustomer',ascending=False)
expanded = pd.merge(expanded, df['av_SalesPerCustomer'], on=['Store'])

#create sales per customer for each store and day
#expanded['SalesPerCustomer']=expanded['Sales']/expanded['Customers']


In [104]:
#create av sales per day of the week
df=expanded.groupby(by=['Store','dayofweek']).agg({'Sales':'sum','Customers':'sum'})
df['av_SalesPerCustomer_dayofweek']=df['Sales']/df['Customers']
expanded = pd.merge(expanded, df['av_SalesPerCustomer_dayofweek'], on=['Store','dayofweek'])


In [105]:
#create av sales per day of month
df=expanded.groupby(by=['Store','dayofmonth']).agg({'Sales':'sum','Customers':'sum'})
df['av_SalesPerCustomer_dayofmonth']=df['Sales']/df['Customers']
expanded = pd.merge(expanded, df['av_SalesPerCustomer_dayofmonth'], on=['Store','dayofmonth'])


In [106]:
#drop if any other values null
data_new=expanded
data_new.set_index('Date',inplace=True)
data_new=data_new.dropna(axis = 0, how ='any')
data_new=data_new[data_new['Sales'] >0 ]
#expanded_new.to_csv('data/train_all_10_Oct.csv')

In [107]:
data_new.shape

(216185, 86)